In [1]:
# Locate project root dir, enable package imports from src/
import sys
from pathlib import Path

PROJ_ROOT = Path().resolve().parents[1]
sys.path.append(str(PROJ_ROOT / "src"))
print(PROJ_ROOT)

/Users/alexandre/Documents/GitHub/price_predictor


In [2]:
import os
import pandas as pd
import numpy as np
import joblib  

# Load standardized paths, ml utilities and custom transformer
from project_utils.paths import CLEANED_DATA_DIR, TEST_PREDICTIONS_DIR
import project_utils.ml_utils as ml_utils

# Model and BaggingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
# Cross-validation
from sklearn.model_selection import KFold, cross_val_score

2025-12-17 13:32:03.848 | INFO     | project_utils.paths:<module>:19 - PROJ_ROOT path is: /Users/alexandre/Documents/GitHub/price_predictor


Note: The model present in this notebook was trained without removing outliers. This decision was based on empirical observation: retaining outliers resulted in more robust models and improved generalization performance, whereas outlier removal consistently degraded validation stability and predictive accuracy.

# Bagging regressor

To enhance the diversity of the stacking ensemble, and given that the inclusion of heterogeneous base learners is known to improve the generalization performance of stacked architectures [1], a bagged regressor was additionally incorporated. The Bagging Regressor is an ensemble meta-estimator that fits multiple instances of a base regressor on different bootstrap samples of the training data and aggregates their predictions to produce a final output. By averaging across these independently KNN trained models, the bagging procedure effectively reduces the variance associated with individual estimators, resulting in a more stable and robust predictive model.

In [3]:
df_train = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_train_data.csv"), index_col="carID"
)
X_test = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_test_data.csv"), index_col="carID"
)

X = df_train.drop("price", axis=1).copy()
y = df_train["price"].copy()

In [4]:
preprocessing_pipeline = joblib.load(
    os.path.join(PROJ_ROOT, "src", "project_utils", "full_preprocessor.joblib")
)
# --- Base estimator: KNN (diversity vs tree models) ---
knn = KNeighborsRegressor(
    n_neighbors=25,
    weights="distance",
    p=2,  # Euclidean
)

# --- Bagging wrapper ---
bagged_knn = BaggingRegressor(
    estimator=knn,
    n_estimators=50,
    bootstrap=True,
    max_samples=0.7,
    random_state=0,
    n_jobs=-1,
)

# --- Log-transform target ---
log_bagged_knn = TransformedTargetRegressor(
    regressor=bagged_knn,
    func=np.log1p,
    inverse_func=np.expm1,
)

pipeline = Pipeline(
    [
        ("preprocessor", preprocessing_pipeline),
        ("regressor", log_bagged_knn),
    ]
)

# --- K-Fold CV ---#
cv = KFold(n_splits=10, shuffle=True, random_state=0)

scores = cross_val_score(
    pipeline, X, y, scoring="neg_mean_absolute_error", cv=cv, n_jobs=-1
)
pipeline.fit(X, y)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('drop_filtered_features', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('group_imputer', ...), ('column_tra

In [5]:
# Compute evaluation metrics
y_pred = pipeline.predict(X)
mae = ml_utils.linear_evaluation_metrics(y, y_pred, verbose=False)[1]
print("Training MAE:", round(mae, 2))
mae_scores = -scores
print("CV MAE mean:", round(mae_scores.mean(), 2))
print("CV MAE std :", round(mae_scores.std(), 2))

Training MAE: 1485.79
CV MAE mean: 1735.47
CV MAE std : 42.47


In [6]:
joblib.dump(pipeline, os.path.join(PROJ_ROOT, "src", "models", "KNN_model.joblib"))

['/Users/alexandre/Documents/GitHub/price_predictor/src/models/KNN_model.joblib']

In [7]:
# -- Export X_test predictions --#
pred_y_test = pipeline.predict(X_test)
# Convert to Series
test_predictions_series = pd.Series(data=pred_y_test, name="price", index=X_test.index)

# Export CSV file
test_predictions_series.to_csv(
    os.path.join(TEST_PREDICTIONS_DIR, "bg_test_predictions.csv")
)